<center>
    <tr>
    <td><img src="images/Quansight_Logo_Lockup_1.png" width="25%"></img></td>
    </tr>
</center>

# Noise Removal: Median and Bilateral Filtering

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

## Median Filtering

To see median filtering, let's start with a simple grayscale image.

In [ ]:
im = cv.imread('data/bambi.jpg',0).astype(np.uint8)

plt.figure(figsize=(10,10))
plt.imshow(im, cmap='gray');

#### Salt-and-pepper noise

Let's add some *salt-and-pepper* (or *impulse*) noise to the image.  This noise can be caused by sharp and sudden disturbances in the image signal. It presents itself as sparsely occurring white and black pixels.

Check [here](https://en.wikipedia.org/wiki/Salt-and-pepper_noise) for more information.

In [ ]:
black = 10
white = 250
mask = np.empty(im.shape, np.uint8)
mask = cv.randu(mask, 0, 255)
im[mask < black] = 0
im[mask > white] = 255

plt.figure(figsize=(10,10))
plt.imshow(im, cmap='gray');

#### Gaussian smoothing with salt-and-pepper noise

The following example supports the assertion that Gaussian smoothing fails to get rid of the salt-and-pepper noise.

In [ ]:
half_width = 2
sigma = 1

plt.figure(figsize=(10,10))
plt.imshow(cv.GaussianBlur(im, (2*half_width+1, 2*half_width+1), sigma), cmap='gray');

### Median filtering as a nonlinear strategy

Median filtering is widely used in digital image processing and signal processing to get rid of *speckle* and *salt-and-pepper* noise while preserving edges.

From Wikipedia article on [median filtering](https://en.wikipedia.org/wiki/Median_filter)

> For small to moderate levels of Gaussian noise, the median filter is demonstrably better than Gaussian blur at removing noise whilst preserving edges for a given, fixed window size.  However, its performance is not that much better than Gaussian blur for high levels of noise, whereas, for speckle noise and salt-and-pepper noise (impulsive noise), it is particularly effective.  Because of this, median filtering is very widely used in digital image processing.

*Aside*: Speckle is a granular interference that inherently exists in and degrades the quality of the active radar, synthetic aperture radar (SAR), medical ultrasound and optical coherence tomography images.  See [here](https://en.wikipedia.org/wiki/Speckle_(interference)) for more information.

<center>
    <tr>
        <td><img src="images/sar-image.png" width="75%"></img></td>
    </tr><br/>
    (Image from Meskine, Fatiha & Miloud, chikr el-mezouar & Taleb, Nasreddine. (2010). A Rigid Image Registration Based on the Nonsubsampled Contourlet Transform and Genetic Algorithms. Sensors. 10. 8553-8571.)     
</center>

### Median filtering in 1D

Replace a value at location $i$ with the median of its neighbourhood values.  

(Treat neighbourhood in a similar vein as the filter width in the case of *linear filtering*.)

In [ ]:
x = np.random.rand(10)*255
x = x.astype(np.uint8)
print(f'x = {x}')

### Live coding session

In [ ]:
# %load solutions/10/solution-01.py


In [ ]:
# %load solutions/10/solution-02.py


### Median filtering in 2D

In [ ]:
half_width = 1

plt.figure(figsize=(20,15))
plt.subplot(121)
plt.title('Gaussian smoothing')
plt.imshow(cv.GaussianBlur(im, (2*half_width+1, 2*half_width+1), sigma), cmap='gray')
plt.subplot(122)
plt.title('Median filtering')
plt.imshow(cv.medianBlur(im, 2*half_width+1), cmap='gray');

---

## Bilateral Filtering

From [Wikipedia](https://en.wikipedia.org/wiki/Bilateral_filter):

> A bilateral filter is a non-linear, edge-preserving, and noise-reducing smoothing filter for images. It replaces the intensity of each pixel with a weighted average of intensity values from nearby pixels. This weight can be based on a Gaussian distribution. Crucially, the weights depend not only on Euclidean distance of pixels, but also on the radiometric differences (e.g., range differences, such as color intensity, depth distance, etc.). This preserves sharp edges.

More details on theory of bilateral filters are available from [a short course offered at SIGGRAPH 2008](https://people.csail.mit.edu/sparis/bf_course). We'll look at the ideas at a very high level here.

### Gaussian kernel in 1D - Review

A Gaussian kernel is a kernel with the shape of a Gaussian (Normal distribution) curve. Here is a standard Gaussian, with a mean of $0$ and a 
$\sigma$ of 1.

In [ ]:
np.random.seed(2)
x_gaussian = np.arange(-5, 5, 0.1)
sigma = 1
y_gaussian = 1. / np.sqrt(2 * np.pi) * np.exp(-x_gaussian**2 / (2. * sigma**2))
plt.figure(figsize=(5,5))
plt.plot(x_gaussian, y_gaussian, c='r');

#### Full Width at Half Maximum Measure (FWHM)

When the Gaussian is used for smoothing, it is common to describe the width of the Gaussian in terms of the Full Width at Half Maximum (FWHM).  The FWHM is the width of the kernel, at half of the maximum of the height of the Gaussian.

In the above example, the maximum value for the Gaussian is ~$0.4$.  The $x$ value corresponding to half the maximum value (i.e., 0.2) is roughly $-1.17$ and $1.17$.  This suggests that the FWHM is ~$2 \times 1.17$.

The following Python functions capture the relationship between $\sigma$ and FWHM.

In [ ]:
def sigma2fwhm(sigma):
    return sigma * np.sqrt(8 * np.log(2))

def fwhm2sigma(fwhm):
    return fwhm / np.sqrt(8 * np.log(2))

In [ ]:
sigma = 1
fwhm = sigma2fwhm(sigma)
print(f'fwhm at sigma = {sigma} is {fwhm}')

In [ ]:
fwhm = 2.35
sigma = fwhm2sigma(2.35)
print(f'sigma at fwhm = {fwhm} is {sigma}')

#### Gaussian smoothing in 1D

Let's first study the effect of Gaussian smoothing in 1D and its effect on edges. We'll consider a 1D step function with some random noise.

In [ ]:
np.random.seed(2)

In [ ]:
N = 2*10 # The trick to ensure N is always even
x = np.arange(N)
x_max = np.max(x)
y_first = np.random.randint(low=10, high=20, size=N//2)
y_second = np.random.randint(low=200, high=250, size=N//2)
y = np.hstack([y_first, y_second])

plt.figure(figsize=(5,5))
plt.title('One dimensional grayscale image')
plt.ylabel('Pixel intensities')
plt.xlabel('Pixel $x$ locations')
plt.ylim(0, 255)
plt.xticks(x)
plt.bar(x, y);

#### Computing Smoothed Values

In order to smooth data using a Gaussian kernel, we need to slide the kernel over the signal and recompute corresponding values.  Consider the signal above. Say we want to recompute the value at pixel location = 5.  We will need to place a Gaussian at this location to compute the new value at location 5.

In [ ]:
scale = 200 # So we can actually see it

FWHM = 8 
sigma = fwhm2sigma(FWHM)
x_position = 15 
kernel_at_pos = np.exp(-(x - x_position) ** 2 / (2 * sigma ** 2))
kernel_at_pos = kernel_at_pos / np.sum(kernel_at_pos) 

plt.figure(figsize=(5,5))
plt.title('One dimensional grayscale image')
plt.ylabel('Pixel intensities')
plt.xlabel('Pixel $x$ locations')
plt.ylim(0, 255)
plt.xticks(x)
plt.bar(x, y)
plt.bar(x, scale*kernel_at_pos, alpha=0.5, color='r');

In [ ]:
x_position_min = max(x_position - FWHM, 0)
x_position_max = min(x_position + FWHM, x_max)+1

print('x: {}'.format(x_position))
print('x min: {}'.format(x_position_min))
print('x max: {}'.format(x_position_max))

relevant_y = y[x_position_min:x_position_max]
relevant_k = kernel_at_pos[x_position_min:x_position_max]
smoothed_y_at_position = np.sum(relevant_y * relevant_k)

np.set_printoptions(precision=3, suppress=True)
print('Value of signal around {}:'.format(x_position), relevant_y)
print('Value of kernel around {}:'.format(x_position), relevant_k)
print('Old y at {} = {:.2f}'.format(x_position, y[x_position]))
print('New y at {} = {:.2f}'.format(x_position, smoothed_y_at_position))

#### Sliding windows

In [ ]:
smoothed_y = np.zeros(y.shape)
for x_position in x:
    kernel = np.exp(-(x - x_position) ** 2 / (2 * sigma ** 2))
    kernel = kernel / np.sum(kernel)
    smoothed_y[x_position] = np.sum(y * kernel)

plt.figure(figsize=(12,5))
plt.subplot(121)
plt.title('One dimensional grayscale image')
plt.xlabel('Pixel locations')
plt.ylabel('Pixel intensities')
plt.xticks(x)
plt.ylim(0,255)
plt.bar(x, y, alpha=1)
plt.subplot(122)
plt.title('One dimensional grayscale image (Smoothed)')
plt.xlabel('Pixel locations')
plt.ylabel('Pixel intensities')
plt.xticks(x)
plt.ylim(0,255)
plt.bar(x, smoothed_y, alpha=1, color='r');

### Bilateral filtering in 1D

In [ ]:
x_position = 9 # Position in array is 10 (0 based) 

FWHM1 = 8 # The value falls below the half of maximum value after 2
sigma1 = fwhm2sigma(FWHM1)
kernel1_at_pos = np.exp(-(x - x_position) ** 2 / (2 * sigma1 ** 2))
kernel1_at_pos = kernel1_at_pos / np.sum(kernel1_at_pos) # Sum to 1

FWHM2 = 60 # The value falls below the half of maximum value after 2
sigma2 = fwhm2sigma(FWHM2)
kernel2_at_pos = np.zeros(kernel1_at_pos.shape)
kernel2_at_pos = np.exp(-(y[x] - y[x_position]) ** 2 / (2 * sigma2 ** 2))
kernel2_at_pos = kernel2_at_pos / np.sum(kernel2_at_pos)

scale = 200 # So we can actually see it

plt.figure(figsize=(12,5))
plt.subplot(121)
plt.title('Nearness')
plt.ylabel('Pixel intensities')
plt.xlabel('Pixel $x$ locations')
plt.ylim(0, 255)
plt.xticks(x)
plt.bar(x, y, alpha=.5)
plt.bar(x, scale*kernel1_at_pos, alpha=0.5, color='r')
plt.subplot(122)
plt.title('Same intensities')
plt.ylabel('Pixel intensities')
plt.xlabel('Pixel $x$ locations')
plt.ylim(0, 255)
plt.xticks(x)
plt.bar(x, y, alpha=.5)
plt.bar(x, scale*kernel2_at_pos, alpha=0.5, color='g');

#### Sliding window

In [ ]:
smoothed_y = np.zeros(y.shape)
for x_position in x:
    kernel1 = np.exp(-(x - x_position) ** 2 / (2 * sigma1 ** 2))
    kernel2 = np.exp(-(y[x] - y[x_position]) ** 2 / (2 * sigma2 ** 2))
    kernel = kernel1 * kernel2
    kernel = kernel / np.sum(kernel)
    smoothed_y[x_position] = np.sum(y * kernel)

plt.figure(figsize=(12,5))
plt.subplot(121)
plt.title('One dimensional grayscale image')
plt.xlabel('Pixel locations')
plt.ylabel('Pixel intensities')
plt.xticks(x)
plt.ylim(0,255)
plt.bar(x, y, alpha=1)
plt.subplot(122)
plt.title('One dimensional grayscale image (Smoothed)')
plt.xlabel('Pixel locations')
plt.ylabel('Pixel intensities')
plt.xticks(x)
plt.ylim(0,255)
plt.bar(x, smoothed_y, alpha=1, color='r');

## Comparison

(Left to right) 1d input image, Gaussian smoothing, and bilateral filtering.  Note that bilateral filtering preserves edge information.

<center>
    <tr>
        <td><img src="images/1d-image-bilateral.png" width="31%"></img></td>
        <td><img src="images/gaussian-bilateral.png" width="31%"></img></td>
        <td><img src="images/bilateral.png" width="31%"></img></td>
    </tr>
</center>

---
Based on materials from Prof. Faisal Qureshi (Faculty of Science, Ontario Tech University, Oshawa ON, Canada, http://vclab.science.ontariotechu.ca)

<center>
    <tr>
    <td><img src="images/Quansight_Logo_Lockup_1.png" width="25%"></img></td>
    </tr>
</center>